## 7. House prices model

In this assignment, you'll continue working with the house prices data. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

1) Load the houseprices data from Thinkful's database.

2) Reimplement your model from the previous checkpoint.

3) Try OLS, Lasso, Ridge, and ElasticNet regression using the same model specification. This time, you need to do k-fold cross-validation to choose the best hyperparameter values for your models. 

4) Which model is the best? Why?


In [1]:

import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [2]:
#Load the data

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host,
    postgres_port, postgres_db))

prices_df= pd.read_sql_query('select * from houseprices', con=engine)

#close the connection
engine.dispose()

prices_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [3]:
#Dropping columns with NaN values that are >50% of column
prices_df.drop(columns=['id','alley', 'poolqc','fence','fireplacequ', 'miscfeature'], axis=1, inplace=True)

### 1) Clean and explore the data

In [4]:
#Use .mean() to in remaining missing values for each column
numeric_cols=prices_df.select_dtypes(include=[np.number]).columns
names=prices_df[numeric_cols]

for name in names:
    prices_df["{}".format(name)]= prices_df["{}".format(name)]\
        .transform(lambda x : x.fillna(x.mean()))

In [5]:
prices_df.dropna(inplace=True)

In [6]:
#Look at correlations among feastures and target variable
numeric_columns = prices_df.select_dtypes(['int64', 'float64']).columns
non_numeric_columns = prices_df.select_dtypes(['object']).columns

np.abs(prices_df[numeric_columns].iloc[:,1:].corr().loc[:,"saleprice"]).sort_values(ascending=False)

saleprice        1.000000
overallqual      0.783546
grlivarea        0.711706
garagecars       0.640154
garagearea       0.607535
firstflrsf       0.604714
totalbsmtsf      0.602042
fullbath         0.569313
totrmsabvgrd     0.551821
yearbuilt        0.504297
yearremodadd     0.501435
garageyrblt      0.481730
masvnrarea       0.465811
fireplaces       0.445434
bsmtfinsf1       0.359677
lotfrontage      0.327831
openporchsf      0.322786
secondflrsf      0.311354
wooddecksf       0.305983
halfbath         0.258175
lotarea          0.254757
bsmtfullbath     0.209695
bsmtunfsf        0.191689
bedroomabvgr     0.169266
enclosedporch    0.127385
kitchenabvgr     0.111408
overallcond      0.108627
screenporch      0.096624
poolarea         0.091881
threessnporch    0.042159
mosold           0.041310
bsmtfinsf2       0.031226
bsmthalfbath     0.030175
yrsold           0.020451
miscval          0.016990
lowqualfinsf     0.009992
Name: saleprice, dtype: float64

In [8]:
#Remove the outliers

import scipy.stats as stats
from scipy.stats.mstats import winsorize

def using_mstats(s):
    return winsorize(s, limits=[0.05, 0.05])

prices = prices_df.apply(using_mstats, axis=0)
prices.quantile([0, 0.05, 0.25, 0.5, 0.75, 0.95, 1])

,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,...,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice
0.00,20.0,36.000000,3230.00,4.0,5.0,1918.00,1950.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,2.0,2006.0,100000.0
0.05,20.0,36.000000,3303.10,4.0,5.0,1918.85,1950.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,2.0,2006.0,100000.0
0.25,20.0,60.000000,7744.00,5.0,5.0,1956.00,1968.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,5.0,2007.0,135000.0
0.50,50.0,70.049958,9600.00,6.0,5.0,1976.00,1994.5,0.0,413.0,0.00,...,6.0,28.0,0.00,0.0,0.0,0.0,0.0,6.0,2008.0,168500.0
0.75,70.0,80.000000,11760.75,7.0,6.0,2001.00,2004.0,174.0,733.0,0.00,...,174.5,70.0,0.00,0.0,0.0,0.0,0.0,8.0,2009.0,220000.0
0.95,160.0,105.000000,17508.85,8.0,8.0,2007.00,2007.0,464.3,1280.3,412.35,...,342.3,172.3,180.45,0.0,168.0,0.0,0.0,11.0,2010.0,335000.0
1.00,160.0,105.000000,17542.00,8.0,8.0,2007.00,2007.0,466.0,1282.0,420.00,...,344.0,174.0,183.00,0.0,168.0,0.0,0.0,11.0,2010.0,335000.0


### 4) Build Linear Regression Model

In [11]:
import statsmodels.api as sm

# Y is the target variable
Y = prices['saleprice']
# X is the feature set which includes lot size, overallcond, full & half bath, bedroomabvgr and totalbsmtsf
X = prices[['grlivarea','overallqual', 'firstflrsf', 'garagecars', 'totalbsmtsf']]


# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     1114.
Date:                Thu, 05 Sep 2019   Prob (F-statistic):               0.00
Time:                        22:05:54   Log-Likelihood:                -15608.
No. Observations:                1338   AIC:                         3.123e+04
Df Residuals:                    1332   BIC:                         3.126e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -9.743e+04   4247.854    -22.936      0.000   -1.06e+05   -8.91e+04
grlivarea      48.6028      2.368     20.529      0.000      43.958      53.247
overallqual  2.058e+04    912.627     22.552      0.000    1.88e+04    2.24e+04
firstflrsf     -3.8568      5.422     -0.711      0.477     -14.494       6.781
garagecars    1.69e+04   1645.459     10.268      0.000    1.37e+04    2.01e+04
totalbsmtsf    48.8495      5.337      9.153      0.000      38.380      59.319
==============================================================================
Omnibus:                       97.000   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              236.884
Skew:                          -0.414   Prob(JB):                     3.64e-52
Kurtosis:                       4.888   Cond. No.                     1.26e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.26e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
#Split the data into training and test data

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))

#We fit an OLS model using sklearn
lrm = LinearRegression()
lrm.fit(X_train, y_train)


# We are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

The number of observations in training set is 1070
The number of observations in test set is 268
R-squared of the model in the training set is: 0.8107065686089059
-----Test set statistics-----
R-squared of the model in the test set is: 0.7614693855972614
Mean absolute error of the prediction is: 21777.23882607024
Mean squared error of the prediction is: 917617257.2778245
Root mean squared error of the prediction is: 30292.197960495116
Mean absolute percentage error of the prediction is: 13.314772656354673


In [24]:
lrm.coef_

array([7.76582049e+00, 2.18061497e+04, 1.64522644e+04, 4.16626057e+01])

In [27]:
from sklearn.linear_model import Ridge

# Fitting a ridge regression model. Alpha is the regularization
# parameter (usually called lambda). As alpha gets larger, parameter
# shrinkage grows more pronounced.
ridgeregr = Ridge(alpha=10**3) 
ridgeregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


R-squared of the model on the training set is: 0.7713658523118934
-----Test set statistics-----
R-squared of the model on the test set is: 0.7338084125776656
Mean absolute error of the prediction is: 22671.942397902352
Mean squared error of the prediction is: 1024027858.9502025
Root mean squared error of the prediction is: 32000.43529313629
Mean absolute percentage error of the prediction is: 13.635249953484932


In [25]:
ridgeregr.coef_

array([2.29562959e-27, 6.51885346e-30, 2.94191197e-30, 3.69745882e-27])

In [32]:
from sklearn.linear_model import Lasso

lassoregr = Lasso(alpha=10**3) 
lassoregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


R-squared of the model on the training set is: 0.8097268423829691
-----Test set statistics-----
R-squared of the model on the test set is: 0.7600554901655375
Mean absolute error of the prediction is: 21755.496090961493
Mean squared error of the prediction is: 923056453.631654
Root mean squared error of the prediction is: 30381.844144680454
Mean absolute percentage error of the prediction is: 13.284704874103506


In [33]:
lassoregr.coef_

array([7.43128483e+00, 2.12977492e+04, 1.28578740e+04, 4.45204074e+01])

In [35]:
from sklearn.linear_model import ElasticNet

elasticregr = ElasticNet(alpha=10**4, l1_ratio=0.5) 
elasticregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(elasticregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


R-squared of the model on the training set is: 0.6738383878597174
-----Test set statistics-----
R-squared of the model on the test set is: 0.6451183318992809
Mean absolute error of the prediction is: 27303.0404149347
Mean squared error of the prediction is: 1365214875.0639458
Root mean squared error of the prediction is: 36948.814257888516
Mean absolute percentage error of the prediction is: 16.43282548699436


In [36]:
elasticregr.coef_

array([12.90328904,  3.15190333,  0.59300258, 72.65265156])